Import Statements

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

Loading the Dataset

In [32]:
data = pd.read_csv('neza_combined_data.csv', index_col=0)
print(data)

         Location  Number of garages  Presence of open sewage  \
0       Hyderabad               4000                        1   
1   Visakhapatnam               2000                        1   
2      Vijayawada               1500                        1   
3        Itanagar                500                        1   
4        Guwahati               2500                        0   
..            ...                ...                      ...   
69       Haridwar                500                        0   
70      Rishikesh                200                        0   
71        Kolkata               5000                        0   
72         Howrah               1500                        0   
73       Durgapur               1000                        0   

    Number of industries Population that uses borehole of water  \
0                     10                                 22.00%   
1                     18                                 22.00%   
2                 

Defining the feature and target variables


In [37]:
y = data['Prevalence of high BLLs']
X = data.drop('Prevalence of high BLLs', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

print("Training data:", X_train.shape)
print("Testing data:", X_test.shape)

print(X_train.head())
print(X_test.head()) 

Training data: (59, 5)
Testing data: (15, 5)
      Location  Number of garages  Presence of open sewage  \
69    Haridwar                500                        0   
18    Vadodara               2000                        1   
70   Rishikesh                200                        0   
59  Coimbatore               2000                        1   
35   Kozhikode               1000                        1   

    Number of industries Population that uses borehole of water  
69                     3                                 40.00%  
18                     1                                 11.00%  
70                    18                                 10.00%  
59                    18                                 16.00%  
35                     1                                 68.00%  
      Location  Number of garages  Presence of open sewage  \
45      Aizawl                200                        0   
64    Agartala                500                        0   
